### Dependencies

In [1]:
import sys
sys.path.append('/home/jovyan/farsite-devAPI/src/')
sys.path.append('/home/jovyan/python-helper/src/')

import geopandas as gpd
import pandas as pd
import numpy as np

from shapely.geometry import MultiPolygon, Polygon

import uuid
import random

import farsiteutils_v2 as futils
from kalmanutils import calculate_uncertainties_observed, interpolate_perimeter
from kalmanutils import State, sample_geometry, calculate_max_area_geom, calculate
from kalmanutils import calculate_rms, calculate_area_diff, align_vertices, interpolate_geometries
from kalmanutils import update_EnKF, fill_zeros, create_ensemble_matrix

from loggers import TimeEstimator

from matplotlib import pyplot as plt

import datetime
import pickle

### Default Values

In [2]:
SELECTED_FIRE_DEFAULT = 'Maria2019'
DATA_PATH = '/home/jovyan/data/'    # Folder to keep all the data while running the simulations

### Select a fire

```selected_fire``` variable holds the name of the fire selected. ```selected_fire = 'Maria2019'``` for now

In [3]:
fp = futils.FilePaths(DATA_PATH)
usr = futils.User(fp)

unique_desc = usr.db.gdfignitionAll['description'].unique()
print(f'Available fires are {unique_desc}')

selected_fire = SELECTED_FIRE_DEFAULT
print(f'{selected_fire} is selected.\n')
if (unique_desc == selected_fire).any():
    print(f'{selected_fire} is found!')
else:
    raise ValueError(f'{selected_fire} not found in possible fire descriptions: {unique_desc}')

Database interaction not yet implemented. Use pickle file for dataframes instead!


AttributeError: 'Database_v2' object has no attribute 'gdfignitionAll'

### Selecting the observations
Wind speed and Wind direction is read from the values used during the fire

In [4]:
objectidlst = [21231, 21232, 21234, 21235, 21236]

# objectidlst = [21231, 21232, 21234, 21235]
wdmaria = [37, 48, 48, 44, 16]
wsmaria = [11, 18, 18, 25, 17]
dtar = [datetime.timedelta(minutes=21), 
        datetime.timedelta(minutes=33), 
        datetime.timedelta(minutes=25), 
        datetime.timedelta(minutes=26)]

gdfmaria = gpd.read_file('/home/jovyan/farsite-devAPI/data/maria-wgs84/maria-wgs84.shp')
# gdfmaria[gdfmaria['objectid'].isin(objectidlst)][['objectid', 'YYYYMMDD', 'windspeed', 'winddirec']]

gdfobserved = usr.db.gdfignition[usr.db.gdfignition['objectid'].isin(objectidlst)]
# gdfobserved

### Starting calculation - 0 $\rightarrow$ 1

In [5]:
# ws = 10
# wd = 60 
######### wd and ws to be chosen from the runs during the live fire
nsamples = 2000
vertex_count = 100

data = {'calculations': [],
        'parameters': {'nsamples': nsamples,
                       'vertex_count': vertex_count},
       }

In [ ]:
%%time

for upix in range(1,len(gdfobserved)):
    print('------------------------------------')
    print('------------------------------------')
    print(f'Calculating {upix-1}-->{upix}')
    print('------------------------------------')
    print('------------------------------------')
    
    # Generate samples of (ws, wd)
    wsar = np.random.normal(wsmaria[upix-1], np.sqrt(wsmaria[upix-1]), nsamples)
    wdar = np.random.normal(wdmaria[upix-1], 30, nsamples)

    compareidx = {}
    # Run for the first ensemble 0->1

    usr_tmp = futils.User(fp)
    if upix == 1:    
        igniteidx = gdfobserved.index[upix-1]
    else:
        igniteidx = updateidx
        usr_tmp.db.gdfignition.loc[igniteidx] = usr.db.gdfignition.loc[igniteidx]
        
    # referenceidx = gdfreference.index[upix]
    compareidx[igniteidx] = []
    
    time_estimator = TimeEstimator(len(wsar))
    for i, (wsflt, wdflt) in enumerate(zip(wsar, wdar)):
        wsint = round(wsflt)
        wdint = np.fmod(round(wdflt)+360, 360)

        print(time_estimator.info_str(i))
        # print(f'\nCalculating for wd={wdint}, ws={wsint}')

        # calculate compareidx for the updated
        compareidx[igniteidx].append(uuid.uuid4().hex)
        calculate(igniteidx, compareidx[igniteidx][-1], usr_tmp, label='Updated', windspeed = wsint, winddirection = wdint, dt=dtar[upix-1])    


    # Calculate the ensemble matrix for the deviated simulations
    Xt, aligned_geom, vertex_count = create_ensemble_matrix(usr_tmp.db.gdfsimulation, nsamples=nsamples, vertex_count=vertex_count)

    # Calculate ensemble matrix for the observations
    geom = gdfobserved.iloc[upix]['geometry']

    uncertainties = np.zeros(nsamples) + 100
    interpolate_geom = interpolate_geometries([geom], vertex_count)[0]
    aligned_vertices = align_vertices([aligned_geom, interpolate_geom])[1]
    st = State(Polygon(aligned_vertices))

    aligned_geoms_lst = []
    for i in range(nsamples):
        newgeom = sample_geometry(st, uncertainties).buffer(0)
        interpolate_geom = interpolate_geometries([newgeom], vertex_count)[0]
        aligned_vertices = align_vertices([aligned_geom, interpolate_geom])[1]
        aligned_geoms_lst.append(Polygon(aligned_vertices))

    Y, aligned_geom, vertex_count = create_ensemble_matrix(gpd.GeoDataFrame(geometry=aligned_geoms_lst), nsamples, vertex_count, aligned_geom, observed=True)


    X = update_EnKF(Xt, Y, aligned_geom)
    ymean = X[1:-2:2,:].mean(axis=1)
    xmean = X[:-2:2,:].mean(axis=1)

    Xgeom = Polygon(zip(xmean, ymean))

    updateidx = uuid.uuid4().hex
    filepath = f'/home/jovyan/data/ignitions/Maria{updateidx}.shp'
    gpd.GeoDataFrame({'FID': [0], 'geometry':Xgeom}, 
                 crs='EPSG:5070').to_file(filepath)

    gdfupdate = gpd.GeoDataFrame(index=[updateidx], data = {'filetype': 'Ignition',
                                          'objectid': f'Update_{upix}',
                                          'filepath': filepath,
                                          'datetime': gdfobserved.iloc[upix]['datetime'],
                                          'description': 'Maria2019',
                                          'geometry': Xgeom})

    usr.db.gdfignition = pd.concat([usr.db.gdfignition, gdfupdate])

    data['calculations'].append({'igniteidx': igniteidx,
                                  'updateidx': updateidx,
                                  'Xt': Xt, 'X': X, 'Y': Y, 'aligned_geom': aligned_geom,
                                  'wdar': wdar, 'wsar': wsar})
    data['gdfignition'] = usr.db.gdfignition

    with open('/home/jovyan/data/output_Maria_2.pkl', 'wb') as f:
        pickle.dump(data, f)

In [ ]:
upix = 1

print('------------------------------------')
print(f'Calculating {upix-1}-->{upix}')

# Generate samples of (ws, wd)
wsar = np.random.normal(wsmaria[upix-1], np.sqrt(wsmaria[upix-1]), nsamples)
wdar = np.random.normal(wdmaria[upix-1], 30, nsamples)

compareidx = {}
# Run for the first ensemble 0->1
usr_tmp = futils.User(fp)
if upix == 1:    
    igniteidx = gdfobserved.index[upix-1]
else:
    igniteidx = updateidx
    usr_tmp.db.gdfignition.loc[igniteidx] = usr.db.gdfignition.loc[igniteidx]

# referenceidx = gdfreference.index[upix]
compareidx[igniteidx] = []

wsflt = wsar[0]
wdflt = wdar[0]

wsint = round(wsflt)
wdint = round(wdflt)
compareidx[igniteidx].append(uuid.uuid4().hex)


In [ ]:
gdfobserved['datetime'].diff()

In [ ]:
calculate(igniteidx, compareidx[igniteidx][-1], usr_tmp, label='Updated', windspeed = wsint, winddirection = wdint, dt=datetime.timedelta(minutes=21))

In [ ]:
dtar